In [ ]:
from google.colab import files
uploaded = files.upload()


Saving kaggle.json to kaggle.json


In [ ]:
import os
import json

# Save uploaded file to the correct location
os.makedirs("/root/.kaggle", exist_ok=True)

# This will read kaggle.json from your upload
with open("kaggle.json") as f:
    kaggle_token = json.load(f)

with open("/root/.kaggle/kaggle.json", "w") as f:
    json.dump(kaggle_token, f)

os.chmod("/root/.kaggle/kaggle.json", 600)


## 🌿 **RE-Psi0 Amazon Discovery: Earth Engine Integration**

This notebook implements a Recursive Emergence (RE-psi0) driven system for identifying potential archaeological sites across the Amazon biome using open-source satellite and Lidar data.

We will:
- Tile the Amazon basin into grid points
- Extract NDVI (vegetation), GEDI canopy height, and elevation data for each point
- Calculate ψ⁰ contradictions and φ⁰ resonance scores
- Visualize a heatmap of high-potential zones

**Focus Region:** Brazil + surrounding Amazonian zones (Bolivia, Peru, Colombia, etc.)

🔗 Project: `amazon-discovery-research`


## 🔐 **Step 1: Authenticate and Initialize Earth Engine**

We connect this notebook to your GCP project and activate access to Earth Engine.


In [ ]:
import ee
ee.Authenticate()  # Follow browser login
ee.Initialize(project='amazon-discovery-research')

## 🗺️ **Step 2: Generate Spatial Grid over the Amazon**

We tile the Amazon biome into 5 km resolution grid points that will be used for feature extraction.


In [ ]:
import numpy as np
import pandas as pd

# Define Amazon bounding box and 5km grid resolution (~0.05 degrees)
lat_min, lat_max = -15.5, 5.0
lon_min, lon_max = -75.0, -45.0
grid_spacing = 0.05  # ~5km at equator

# Create latitude and longitude points
lat_points = np.arange(lat_min, lat_max, grid_spacing)
lon_points = np.arange(lon_min, lon_max, grid_spacing)

# Create all combinations of lat/lon
grid_coords = [{"Latitude": lat, "Longitude": lon}
               for lat in lat_points for lon in lon_points]

# Convert to DataFrame
df_grid = pd.DataFrame(grid_coords)
df_grid.head()


,Latitude,Longitude
0,-15.5,-75.00
1,-15.5,-74.95
2,-15.5,-74.90
3,-15.5,-74.85
4,-15.5,-74.80


In [ ]:
df_grid.to_csv("amazon_grid_5km.csv", index=False)

##🛰️ **Step 3 — NDVI + GEDI Extraction (GEE)**

In [ ]:
import ee
import time

# Define NDVI extraction from Sentinel-2
def get_ndvi(lat, lon):
    try:
        point = ee.Geometry.Point([lon, lat])
        region = point.buffer(2500).bounds()  # 5km buffer

        img = (
            ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
            .filterBounds(region)
            .filterDate("2024-11-01", "2025-05-01")
            .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20))
            .median()
        )

        ndvi = img.normalizedDifference(["B8", "B4"]).rename("NDVI")
        value = ndvi.reduceRegion(
            reducer=ee.Reducer.mean(), geometry=region, scale=10
        ).get("NDVI")

        return value.getInfo() if value else None
    except Exception as e:
        return None

# Define GEDI RH100 canopy height extraction
def get_gedi_height(lat, lon):
    try:
        point = ee.Geometry.Point([lon, lat])
        gedi = ee.ImageCollection("LARSE/GEDI/GEDI02_A_002_MONTHLY")
        img = gedi.sort("system:time_start", False).first()

        height = img.select("rh100").reduceRegion(
            reducer=ee.Reducer.mean(), geometry=point, scale=25
        ).get("rh100")

        return height.getInfo() if height else None
    except Exception as e:
        return None


In [ ]:
import time

# Sample 3 points from your grid for testing
sample_grid = df_grid.sample(3, random_state=42).copy()

results = []
for _, row in sample_grid.iterrows():
    lat, lon = row["Latitude"], row["Longitude"]

    try:
        ndvi = get_ndvi(lat, lon)
        rh100 = get_gedi_height(lat, lon)
    except Exception as e:
        print(f"Failed at ({lat}, {lon}) with error: {e}")
        ndvi, rh100 = None, None

    results.append({
        "Latitude": lat,
        "Longitude": lon,
        "NDVI": ndvi,
        "GEDI_RH100": rh100
    })
    time.sleep(1)  # throttle requests to avoid hitting the API rate limit

df_features = pd.DataFrame(results)
df_features

,Latitude,Longitude,NDVI,GEDI_RH100
0,-11.6,-63.7,0.668387,None
1,-3.5,-55.4,0.381915,None
2,-12.7,-48.7,0.757730,None


# 📐 **Step 4 — φ⁰ Scoring Function**

In [ ]:
# Define normalization functions for 0–5 scaling
def normalize_ndvi(ndvi):
    if ndvi is None:
        return 0
    return min(max((ndvi - 0.3) / (0.8 - 0.3) * 5, 0), 5)  # Typical healthy NDVI range 0.3–0.8

def normalize_gedi(height):
    if height is None:
        return 0
    return min(max((height - 5) / (30 - 5) * 5, 0), 5)  # Normalize 5m–30m canopy as meaningful range


# 🧮 **Apply to Extracted Grid**

In [ ]:
df_features["NDVI_Score"] = df_features["NDVI"].apply(normalize_ndvi)
df_features["GEDI_Score"] = df_features["GEDI_RH100"].apply(normalize_gedi)

# Total φ⁰ score as simple average (weights can be adjusted later)
df_features["φ⁰_Score"] = df_features[["NDVI_Score", "GEDI_Score"]].mean(axis=1)
df_features[["Latitude", "Longitude", "NDVI_Score", "GEDI_Score", "φ⁰_Score"]]


,Latitude,Longitude,NDVI_Score,GEDI_Score,φ⁰_Score
0,-11.6,-63.7,3.683869,0,1.841935
1,-3.5,-55.4,0.819148,0,0.409574
2,-12.7,-48.7,4.577300,0,2.288650


## ✴️ ***Step 4.1: φ⁰ Scoring and Collapse Boost***

We normalize NDVI and GEDI RH100 into 0–5 scales:

- **NDVI_Score**: Higher values = greener vegetation
- **GEDI_Score**: Higher values = taller canopy

We then compute the base φ⁰ score as the average of NDVI and GEDI scores.

Finally, we apply a **ψ⁰ contradiction boost** if:
- NDVI is high (lush vegetation)
- Canopy height is low (sparse structure)
This pattern suggests possible regrowth over cleared zones — a **ψ⁰ tension field** that may indicate hidden archaeological activity beneath.

A boost of +1.0 is added to the φ⁰ score in these cases.


In [ ]:
def apply_collapse_boost(row):
    ndvi_score = row["NDVI_Score"]
    gedi_score = row["GEDI_Score"]

    # Boost if vegetation is lush (NDVI > 3.5) but canopy is sparse (GEDI < 2.5)
    if ndvi_score >= 3.5 and gedi_score <= 2.5:
        return 1.0  # collapse tension detected
    return 0.0

df_features["ψ⁰_Boost"] = df_features.apply(apply_collapse_boost, axis=1)
df_features["φ⁰_Score"] = df_features["φ⁰_Score"] + df_features["ψ⁰_Boost"]


# ** 🗺️ Summary in Markdown**

### **🌀 Step 4.2: Mythic Proximity Overlay (ψ⁰ Injection)**

We embed a symbolic proximity boost based on how close each point is to major myth-zones:

- **Paititi**, **Lost City of Z**, **El Dorado**

**Scoring Logic**:
- < 50 km → +1.0 boost (strong φ⁰ pull)
- < 100 km → +0.5 boost (weak field)
- > 100 km → 0 (no effect)

These overlays function as **ψ⁰ attractors**, allowing us to prioritize zones not only by ecology, but also symbolic memory.


# **📍Define Myth Points (Seed Locations)**

In [ ]:
# Mythic coordinate seeds (you can add more!)
myth_zones = [
    {"name": "Paititi", "lat": -12.5, "lon": -70.5},
    {"name": "Lost City of Z", "lat": -11.5, "lon": -53.2},
    {"name": "El Dorado", "lat": 3.5, "lon": -66.1},
    {"name": "Kuhikugu", "lat": -12.558333,"lon": -53.111111}   # this is a confirmed location
]


# **📏 Boost Function: Myth Proximity → φ⁰**

In [ ]:
from geopy.distance import geodesic

def myth_proximity_boost(lat, lon):
    min_distance_km = min(
        geodesic((lat, lon), (zone["lat"], zone["lon"])).km
        for zone in myth_zones
    )
    if min_distance_km < 50:
        return 1.0
    elif min_distance_km < 100:
        return 0.5
    else:
        return 0.0

df_features["Myth_Boost"] = df_features.apply(
    lambda row: myth_proximity_boost(row["Latitude"], row["Longitude"]), axis=1
)

# Update φ⁰ score with mythic resonance
df_features["φ⁰_Score"] = df_features["φ⁰_Score"] + df_features["Myth_Boost"]


# 🗺️ **Step 5 — Folium Heatmap Code**

In [ ]:
!pip install folium --quiet

import folium
from folium.plugins import HeatMap


# **🔥 Plot φ⁰ Heatmap**

In [ ]:
# 📦 Install Folium if not already installed
!pip install folium --quiet

import folium
from folium.plugins import HeatMap

# ⚠️ Avoid chained assignment warning
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Normalize φ⁰ scores for heatmap intensity
heat_data = df_features[df_features["φ⁰_Score"].notnull()].copy()
heat_data["φ⁰_Intensity"] = heat_data["φ⁰_Score"] / heat_data["φ⁰_Score"].max()

# 🌎 Initialize the base map
m = folium.Map(location=[-5, -65], zoom_start=5, tiles="CartoDB dark_matter")

# 🔥 Add φ⁰ heatmap layer
heatmap = HeatMap(
    data=heat_data[["Latitude", "Longitude", "φ⁰_Intensity"]].values.tolist(),
    min_opacity=0.2,
    max_opacity=0.9,
    radius=12,
    blur=18
)
m.add_child(heatmap)

# 📍 Mythic attractors (ψ⁰ seeds)
myth_zones = [
    {"name": "Paititi", "lat": -12.5, "lon": -70.5},
    {"name": "Lost City of Z", "lat": -11.5, "lon": -53.2},
    {"name": "El Dorado", "lat": 3.5, "lon": -66.1},
    {"name": "Kuhikugu", "lat": -12.558333,"lon": -53.111111}   # this is a confirmed location
]

# 🌀 Add myth markers with labels and popups
for zone in myth_zones:
    folium.CircleMarker(
        location=[zone["lat"], zone["lon"]],
        radius=7,
        color="gold",
        fill=True,
        fill_opacity=0.85,
        popup=folium.Popup(f"{zone['name']} (ψ⁰ Seed)", max_width=150),
        tooltip=zone["name"]
    ).add_to(m)

# 🖼️ Display map
m


## 🌲 Step 6: GEDI Canopy Height (RH100) Visualization

We use GEDI Lidar RH100 values (25m footprint, ISS-based) to assess vegetation height in each φ⁰ candidate zone.

- Areas with **low canopy + high NDVI** are highlighted as ψ⁰ contradictions.
- GEDI provides real lidar-based height data, helping validate potential anthropogenic flattening.

Green = tall canopy  
Orange = sparse canopy (possible anomaly)


In [ ]:
import folium
from folium.plugins import HeatMap
import pandas as pd
import numpy as np
import random

# First, check if we have any valid GEDI data
print("Current data in df_features:")
print(df_features)

# Check if we have any valid GEDI data
has_valid_gedi = df_features["GEDI_RH100"].notnull().any()
print(f"\nValid GEDI data available: {has_valid_gedi}")

# If we have no valid data, create sample data for visualization purposes
if not has_valid_gedi or len(df_features) < 3:
    print("\nGenerating sample data for visualization...")

    # Create a more comprehensive sample dataset for the Amazon basin
    amazon_center_lat, amazon_center_lon = -5, -60
    sample_size = 20

    # Generate random points around the Amazon
    sample_data = []
    for i in range(sample_size):
        # Random offsets within ~500km of Amazon center
        lat_offset = random.uniform(-5, 5)
        lon_offset = random.uniform(-10, 10)

        lat = amazon_center_lat + lat_offset
        lon = amazon_center_lon + lon_offset

        # Generate realistic GEDI canopy heights (5m-35m)
        canopy_height = random.uniform(5, 35)

        # Generate realistic NDVI values (0.3-0.9)
        ndvi = random.uniform(0.3, 0.9)

        sample_data.append({
            "Latitude": lat,
            "Longitude": lon,
            "NDVI": ndvi,
            "GEDI_RH100": canopy_height
        })

    # Convert to DataFrame and use for visualization
    viz_df = pd.DataFrame(sample_data)
    print(f"Created {len(viz_df)} sample points for visualization")
else:
    # Use our real data
    viz_df = df_features.copy()
    print(f"Using {len(viz_df)} real data points for visualization")

# Calculate scores as in original code
viz_df["NDVI_Score"] = viz_df["NDVI"].apply(lambda x: min(max((x - 0.3) / (0.8 - 0.3) * 5, 0), 5) if x is not None else 0)
viz_df["GEDI_Score"] = viz_df["GEDI_RH100"].apply(lambda x: min(max((x - 5) / (30 - 5) * 5, 0), 5) if x is not None else 0)
viz_df["φ⁰_Score"] = viz_df[["NDVI_Score", "GEDI_Score"]].mean(axis=1)

# Apply ψ⁰ contradiction boost
viz_df["ψ⁰_Boost"] = viz_df.apply(lambda row: 1.0 if row["NDVI_Score"] >= 3.5 and row["GEDI_Score"] <= 2.5 else 0.0, axis=1)
viz_df["φ⁰_Score"] = viz_df["φ⁰_Score"] + viz_df["ψ⁰_Boost"]

# Create GEDI visualization
gedi_map = viz_df.copy()
gedi_map["Norm_Height"] = gedi_map["GEDI_RH100"] / gedi_map["GEDI_RH100"].max()

# Create base map
m_gedi = folium.Map(location=[-5, -60], zoom_start=4, tiles="CartoDB positron")

# Add title
title_html = '''
<div style="position: fixed;
            top: 10px; left: 50px; width: 300px; height: 30px;
            z-index:9999; font-size:16px; font-weight: bold;
            background-color: white; border-radius: 5px; padding: 5px;
            border: 2px solid grey;">
    Amazon Canopy Height Analysis
</div>
'''
m_gedi.get_root().html.add_child(folium.Element(title_html))

# Add GEDI markers with improved styling
for _, row in gedi_map.iterrows():
    folium.CircleMarker(
        location=[row["Latitude"], row["Longitude"]],
        radius=3 + row["Norm_Height"] * 10,  # Size based on height
        color="green" if row["Norm_Height"] > 0.5 else "orange",  # Color based on height
        fill=True,
        fill_opacity=0.7,
        popup=folium.Popup(f"<b>Location:</b> {row['Latitude']:.4f}, {row['Longitude']:.4f}<br>"
                          f"<b>Canopy Height:</b> {row['GEDI_RH100']:.1f} m<br>"
                          f"<b>NDVI:</b> {row['NDVI']:.2f}<br>"
                          f"<b>φ⁰ Score:</b> {row['φ⁰_Score']:.2f}",
                          max_width=200),
        tooltip=f"{row['GEDI_RH100']:.1f} m"
    ).add_to(m_gedi)

# Add myth markers
myth_zones = [
    {"name": "Paititi", "lat": -12.5, "lon": -70.5},
    {"name": "Lost City of Z", "lat": -11.5, "lon": -53.2},
    {"name": "El Dorado", "lat": 3.5, "lon": -66.1},
    {"name": "Kuhikugu", "lat": -12.558333,"lon": -53.111111}  # confirmed location
]

# Add myth zones with better styling
for zone in myth_zones:
    # Add outer ring (influence area)
    folium.Circle(
        location=[zone["lat"], zone["lon"]],
        radius=50000,  # 50km
        color="gold",
        weight=1,
        fill=True,
        fill_opacity=0.1,
        tooltip=f"{zone['name']} - 50km zone"
    ).add_to(m_gedi)

    # Add center marker
    folium.CircleMarker(
        location=[zone["lat"], zone["lon"]],
        radius=7,
        color="gold",
        fill=True,
        fill_opacity=0.9,
        popup=folium.Popup(f"<b>{zone['name']}</b><br>ψ⁰ Seed Location", max_width=150),
        tooltip=zone["name"]
    ).add_to(m_gedi)

# Create legend
legend_html = '''
<div style="position: fixed;
            bottom: 50px; left: 50px; width: 150px;
            border:2px solid grey; z-index:9999; font-size:14px;
            background-color:white; padding: 10px;
            border-radius: 5px;">
    <p><b>Legend</b></p>
    <p><i class="fa fa-circle" style="color:green;"></i> Tall Canopy (>15m)</p>
    <p><i class="fa fa-circle" style="color:orange;"></i> Low Canopy (<15m)</p>
    <p><i class="fa fa-circle" style="color:gold;"></i> Mythic Sites</p>
    <p>Size indicates relative height</p>
</div>
'''
m_gedi.get_root().html.add_child(folium.Element(legend_html))

# Display map
m_gedi

Current data in df_features:
   Latitude  Longitude      NDVI GEDI_RH100  NDVI_Score  GEDI_Score  φ⁰_Score  \
0     -11.6      -63.7  0.668387       None    3.683869           0  2.841935   
1      -3.5      -55.4  0.381915       None    0.819148           0  0.409574   
2     -12.7      -48.7  0.757730       None    4.577300           0  3.288650   

   ψ⁰_Boost  Myth_Boost  
0       1.0         0.0  
1       0.0         0.0  
2       1.0         0.0  

Valid GEDI data available: False

Generating sample data for visualization...
Created 20 sample points for visualization
